# User Metadata Construction from Browsing Logs

This notebook constructs a **comprehensive user-level metadata table** by combining:

- Raw device-level logs from all schools, including Term 1 (`MG_ files`) and Term 3 (`Browsing History files`).

Official school metadata from `school_category_3.csv`, including school name, gender, and PSLE cutoff group.

Each user is uniquely identified by a `device_name_actual` within a school.

---
## 0. Imports and Inputs

**Input Variables**

- `RAW_DATA_ROOT` → Path to the folder containing **raw device-level CSV logs** for all schools.  
- `SCHOOL_CATEGORY_FILE` → Path to the official **school metadata CSV** (`school_category_3.csv`).  
- `OUTPUT_FILE_CSV` → Path where the **final user-level metadata CSV** will be saved.  
- `CHUNKSIZE` → Number of rows to read per chunk when processing large CSV files to avoid memory issues (here, 200,000 rows per chunk).

In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# =====================================================
# 0. INPUTS
# =====================================================
RAW_DATA_ROOT = Path("/Users/tdf/Downloads/raw_data")
SCHOOL_CATEGORY_FILE = "/Users/tdf/Downloads/school_category_3.csv"
OUTPUT_FILE_CSV = "/Users/tdf/Downloads/user_metadata.csv"
CHUNKSIZE = 200_000

---
## 1. Load School Metadata

This step loads and cleans the official school metadata, preparing it for merging with device-level data.

- **Load CSV**  
  - Read `school_category_3.csv` into a DataFrame `school_cat`  
  - Use `low_memory=False` to avoid type inference issues on large files  
  - Use `encoding='utf-8'` to handle special characters correctly  

- **Fix mis-encoding for St. Margaret’s Secondary School**  
  - Detect rows where the school name contains the mis-encoded `â€™` and "Margaret"  
  - Replace `â€™` with the correct apostrophe `’`  

- ***Map gender to numeric codes**  
  - Convert textual gender labels to numbers for easier processing:  
    - `girls → 1`  
    - `boys → 2`  
    - `co-ed → 99`  

- ** Keep only relevant columns**  
  - Keep `name` (school name), `gender` (numeric code), and `psle_al_cutoff_pg3_group`  
  - Rename `psle_al_cutoff_pg3_group` → `sch_psle` for clarity  

In [2]:
# =====================================================
# 1. LOAD SCHOOL METADATA
# =====================================================
school_cat = pd.read_csv(SCHOOL_CATEGORY_FILE, low_memory=False, encoding='utf-8')

# Fix mis-encoding for St. Margaret’s
mask_margaret = school_cat['name'].str.contains('â€™', regex=False) & \
                school_cat['name'].str.contains('Margaret', regex=False)
school_cat.loc[mask_margaret, 'name'] = school_cat.loc[mask_margaret, 'name'].str.replace('â€™', '’', regex=False)

# Map gender to numbers
gender_map = {'girls': 1, 'boys': 2, 'co-ed': 99}
school_cat['gender'] = school_cat['gender'].map(gender_map)

# Keep only required columns
school_cat = school_cat[['name', 'gender', 'psle_al_cutoff_pg3_group']]
school_cat.rename(columns={'psle_al_cutoff_pg3_group': 'sch_psle'}, inplace=True)

---
## 2. Collect Term 1 & Term 3 Device Presence

This step scans all raw device-level CSV files to build **sets of unique devices** present in Term 1 and Term 3.

- ** Identify school folders**  
  - List all subfolders in `RAW_DATA_ROOT`, where each folder corresponds to a school.

- **Iterate through CSV files**  
  - For each school folder, find all `.csv` files.  
  - Determine the **term type** based on the filename:  
    - Filenames containing `"MG"` → **Term 1**  
    - Filenames containing `"Browsing History"` → **Term 3**  
    - Skip other files

- **Read device names in chunks**  
  - Use `pd.read_csv` with `chunksize=CHUNKSIZE` to handle large files efficiently  
  - Extract the `device_name_actual` column, drop missing values, and get unique devices

- **Update device sets**  
  - Add Term 1 devices to `term1_devices` set  
  - Add Term 3 devices to `term3_devices` set  

- **Error handling**  
  - If a CSV fails to read, skip it and print a warning

In [3]:
# =====================================================
# 2. SCAN ALL SCHOOL FILES TO BUILD TERM 1 & TERM 3 DEVICE SETS
# =====================================================
term1_devices = set()
term3_devices = set()

school_folders = [f for f in RAW_DATA_ROOT.iterdir() if f.is_dir()]

for school_folder in tqdm(school_folders, desc="Processing schools"):
    csv_files = list(school_folder.glob("*.csv"))
    for csv_file in tqdm(csv_files, desc=f"{school_folder.name}", leave=False):
        filename_lower = csv_file.name.lower()
        if "mg" in filename_lower:             # Term 1
            file_type = "term1"
        elif "browsing history" in filename_lower:  # Term 3
            file_type = "term3"
        else:
            continue

        try:
            for chunk in pd.read_csv(csv_file, usecols=['device_name_actual'],
                                     dtype={'device_name_actual': str}, chunksize=CHUNKSIZE):
                devices = chunk['device_name_actual'].dropna().unique()
                if file_type == "term1":
                    term1_devices.update(devices)
                else:
                    term3_devices.update(devices)
        except Exception as e:
            print(f"Skipping {csv_file.name} due to error: {e}")

print(f"Term 1 devices: {len(term1_devices)}, Term 3 devices: {len(term3_devices)}")


Processing schools: 100%|███████████████████████| 48/48 [03:24<00:00,  4.26s/it]

Term 1 devices: 36281, Term 3 devices: 48594


---
## 3. Build User Metadata 

This step constructs a **user-level metadata table** by combining raw device logs with official school information. Each user is uniquely identified by their `device_name_actual` and school.

- **Load school metadata**  
  - Read `school_category_3.csv` into `school_cat`  
  - Fix encoding issues (e.g., St. Margaret’s) to ensure school names are correct  
  - Normalize school names for safe lookup (`name_norm`)  
  - Map gender to numbers:  
    - Girls → `1`  
    - Boys → `2`  
    - Co-ed → `99`

- **Manual school name mapping**  
  - Maps raw folder names in `raw_data` to official school names  
  - Ensures that folder names match metadata for correct merging

- **Iterate over school folders**  
  - For each school folder:  
    - Lookup the official school name  
    - Fetch school info: `gender` and `sch_psle` from metadata  

- **Read device CSVs in chunks**  
  - Iterate all `.csv` files in the folder  
  - Use `chunksize=CHUNKSIZE` for memory efficiency  
  - Extract unique `device_name_actual` entries  

- **Assign student level**  
  - **Level 1** → devices present in Term 3 but **not** in Term 1 (new Secondary 1 students)  
  - **Level 2** → all other devices (Secondary 2–5 students)  

- **Build user records**  
  - Append a dictionary for each device containing:  
    - `device_name_actual`  
    - `school_name_official`  
    - `gender`  
    - `sch_psle`  
    - `level`

- **Convert to DataFrame**  
  - Combine all user records into `users_df` for further processing and export

In [5]:
# =====================================================
# 3. BUILD USER METADATA (corrected)
# =====================================================
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Load school metadata
school_cat = pd.read_csv("/Users/tdf/Downloads/school_category_3.csv", encoding='utf-8', low_memory=False)

# Fix St. Margaret's mis-encoding
mask_margaret = school_cat['name'].str.contains('â€™', regex=False) & \
                school_cat['name'].str.contains('Margaret', regex=False)
school_cat.loc[mask_margaret, 'name'] = school_cat.loc[mask_margaret, 'name'].str.replace('â€™', '’', regex=False)

# Normalize school names for safe lookup
school_cat['name_norm'] = school_cat['name'].str.strip().str.lower()

# Gender mapping: girls=1, boys=2, co-ed=99
gender_map = {'girls': 1, 'boys': 2, 'co-ed': 99}

# Manual mapping: raw folder names -> official school names
manual_school_mapping = {
    "assumption english sch": "Assumption English School",
    "bendemeer secondary": "Bendemeer Secondary School",
    "boon lay secondary": "Boon Lay Secondary School",
    "bowen secondary": "Bowen Secondary School",
    "broadrick secondary": "Broadrick Secondary School",
    "bukit view secondary": "Bukit View Secondary School",
    "canberra secondary": "Canberra Secondary School",
    "catholic high school": "Catholic High School (Secondary)",
    "chij st joseph’s convent": "CHIJ St. Joseph's Convent",
    "chij st nicholas girls_ school (secondary)": "CHIJ St. Nicholas Girls' School (Secondary)",
    "christ church secondary": "Christ Church Secondary School",
    "chua chu kang secondary": "Chua Chu Kang Secondary School",
    "chung cheng high (main)": "Chung Cheng High School (Main)",
    "chung cheng high school (yishun)": "Chung Cheng High School (Yishun)",
    "dunman high school": "Dunman High School",
    "edgefield secondary": "Edgefield Secondary School",
    "evergreen secondary": "Evergreen Secondary School",
    "fuhua secondary": "Fuhua Secondary School",
    "greendale secondary": "Greendale Secondary School",
    "guangyang secondary": "Guangyang Secondary School",
    "hai sing catholic school": "Hai Sing Catholic School",
    "hillgrove secondary": "Hillgrove Secondary School",
    "kent ridge secondary": "Kent Ridge Secondary School",
    "kuo chuan presbyterian": "Kuo Chuan Presbyterian Secondary School",
    "manjusri secondary": "Manjusri Secondary School",
    "maris stella high school": "Maris Stella High School (Secondary)",
    "mayflower secondary": "Mayflower Secondary School",
    "methodist girls' school (secondary)": "Methodist Girls' School (Secondary)",
    "montfort secondary": "Montfort Secondary School",
    "nan chiau high school": "Nan Chiau High School",
    "ngee ann secondary": "Ngee Ann Secondary School",
    "north vista secondary": "North Vista Secondary School",
    "northland secondary": "Northland Secondary School",
    "orchid park secondary": "Orchid Park Secondary School",
    "outram secondary": "Outram Secondary School",
    "pasir ris secondary": "Pasir Ris Secondary School",
    "raffles girls' school": "Raffles Girls' School",
    "seng kang secondary": "Seng Kang Secondary School",
    "singapore sports school": "Singapore Sports School",
    "st margaret's secondary": "St. Margaret’s Secondary School",
    "swiss cottage secondary": "Swiss Cottage Secondary School",
    "whitley secondary": "Whitley Secondary School",
    "woodgrove secondary school": "Woodgrove Secondary School",
    "woodlands secondary": "Woodlands Secondary School",
    "xinmin secondary": "Xinmin Secondary School",
    "yio chu kang secondary": "Yio Chu Kang Secondary School",
    "yishun secondary": "Yishun Secondary School",
    "yishun town secondary": "Yishun Town Secondary School"
}

# Set raw data root
RAW_DATA_ROOT = Path("/Users/tdf/Downloads/raw_data")
school_folders = [f for f in RAW_DATA_ROOT.iterdir() if f.is_dir()]

# Normalize term1/term3 device lists
term1_devices_norm = set(d.strip().lower() for d in term1_devices)
term3_devices_norm = set(d.strip().lower() for d in term3_devices)

all_users = []

# Iterate over all school folders
for school_folder in tqdm(school_folders, desc="Building user metadata"):
    raw_name = school_folder.name.strip().lower()
    school_name_official = manual_school_mapping.get(raw_name)
    
    if school_name_official is None:
        print(f"Warning: {school_folder.name} not found in mapping. Skipping.")
        continue
    
    # Fetch school info
    school_info = school_cat[school_cat['name_norm'] == school_name_official.strip().lower()].iloc[0]
    gender = gender_map[school_info['gender'].strip().lower()]
    sch_psle = school_info['psle_al_cutoff_pg3_group']

    # Iterate CSV files for the school
    csv_files = list(school_folder.glob("*.csv"))
    for csv_file in csv_files:
        try:
            for chunk in pd.read_csv(
                csv_file,
                usecols=['device_name_actual'],
                dtype={'device_name_actual': str},
                chunksize=CHUNKSIZE,
                low_memory=False
            ):
                for device in chunk['device_name_actual'].dropna().unique():
                    device_norm = device.strip().lower()
                    
                    # Assign level
                    if device_norm in term3_devices_norm and device_norm not in term1_devices_norm:
                        level = 1  # Secondary 1
                    else:
                        level = 2  # Secondary 2-5
                    
                    all_users.append({
                        'device_name_actual': device,
                        'school_name_official': school_name_official,
                        'gender': gender,
                        'sch_psle': sch_psle,
                        'level': level
                    })
        except Exception as e:
            print(f"Skipping {csv_file.name} due to error: {e}")

# Convert to DataFrame
users_df = pd.DataFrame(all_users)

Building user metadata: 100%|███████████████████| 48/48 [03:25<00:00,  4.29s/it]


---
## 4. Final Checks and Export

This step performs **quality control** on the constructed user metadata before exporting:

- **Check for missing values**  
  - Use `isna().sum()` to count any missing entries in the key columns:  
    - `school_name_official` → the official school name of the device  
    - `gender` → numeric gender label (1=Girls, 2=Boys, 99=Co-ed)  
    - `sch_psle` → school PSLE cutoff group  
    - `level` → student level (1=Secondary 1, 2=Secondary 2–5)  
  - Ensures that the dataset is complete and ready for downstream analysis

In [7]:
# =====================================================
# 4. FINAL CHECKS AND EXPORT
# =====================================================
print("\nMissing values summary:")
print(users_df[['school_name_official','gender','sch_psle','level']].isna().sum())


Missing values summary:
school_name_official    0
gender                  0
sch_psle                0
level                   0
dtype: int64


---
## 5. Export User Metadata

This step **saves the final user-level metadata** to a CSV file:

- **Export to CSV**  
  - `users_df.to_csv(OUTPUT_FILE_CSV, index=False)` writes the dataset to the specified path  
  - `index=False` ensures the row index is not included in the CSV 

In [8]:
# =====================================================
# 5. EXPORT
# =====================================================
OUTPUT_FILE_CSV = "/Users/tdf/Downloads/user_metadata.csv"
users_df.to_csv(OUTPUT_FILE_CSV, index=False)

print(f"\nTotal users collected: {len(users_df)}")
print(f"User metadata CSV created: {OUTPUT_FILE_CSV}")


Total users collected: 1075262
User metadata CSV created: /Users/tdf/Downloads/user_metadata.csv
